In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import pandas as pd
import torch
import sys
import os

In [29]:
process_dirs = ["ya_kout"]

for root, dirs, files in os.walk("./data"):
    for dir in dirs:
        if dir in process_dirs:
            print("Processing", dir)
            for root2, dirs2, files2 in os.walk(os.path.join(root, dir)):
                images = []
                image_filenames = []
                for file in files2:
                    img = cv.imread(os.path.join(root2, file))
                    # Find the smallest rect bounds for the image
                    # its a black shape inside
                    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                    
                    # Gaussian blur
                    gray = cv.GaussianBlur(gray, (3, 3), 0)
                    
                    _, thresh = cv.threshold(
                        gray, 100, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
                    )
                    
                    # Manual filter threshold using numpy
                    thresh = np.where(gray < 150, 0, 255).astype(np.uint8)             
                    
                    # Inverse image
                    thresh = cv.bitwise_not(thresh)
                     
                    # Detect countours and find an outer bound
                    contours, hierarchy = cv.findContours(
                        thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE
                    )
                    
                    # Find the biggest contour
                    max_area = 0
                    max_contours = []
                    for contour in contours:
                        area = cv.contourArea(contour)
                        if area > max_area:
                            max_area = area
                            max_contours.append(contour)
                        
                        if max_area == 0:
                            continue
                        
                        area_variance = abs(1 - (area / max_area))
                        print(f"Area variance: {area_variance}, Area: {area}, Max Area: {max_area}")
                        if area_variance < 0.8 and area_variance > 0:
                            max_contours.append(contour)
                            
                    def combine_bounding_boxes(rects):
                        if not rects:
                            return None

                        min_x = float('inf')
                        min_y = float('inf')
                        max_x = float('-inf')
                        max_y = float('-inf')

                        for (x, y, w, h) in rects:
                            min_x = min(min_x, x)
                            min_y = min(min_y, y)
                            max_x = max(max_x, x + w)
                            max_y = max(max_y, y + h)

                        combined_x = min_x
                        combined_y = min_y
                        combined_w = max_x - min_x
                        combined_h = max_y - min_y

                        return (combined_x, combined_y, combined_w, combined_h)
                    
                    bounds = [cv.boundingRect(contour) for contour in max_contours]
                    x, y, w, h = combine_bounding_boxes(bounds)
                    
                    thresh = cv.bitwise_not(thresh)
                    
                    # debug = cv.cvtColor(thresh, cv.COLOR_GRAY2BGR)
                    
                    # # Draw the bounding box
                    # for x_, y_, w_, h_ in bounds:
                    #     cv.rectangle(debug, (x_, y_), (x_ + w_, y_ + h_), (0, 255, 0), 2)

                    # cv.rectangle(debug, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    # plt.imshow(debug)
                    # plt.show()
                    
                    # Crop
                    thresh = thresh[y:y+h, x:x+w]    
                    
                    images.append(thresh)
                    image_filenames.append(file)
                    
                    # plt.imshow(thresh, cmap="gray")
                    # plt.show()
                    
                # create sub dir 
                os.makedirs(os.path.join(root2, "cropped"), exist_ok=True)

                for i, image in enumerate(images):
                    cv.imwrite(os.path.join(root2, "cropped", image_filenames[i]), image)
                

Processing ya_kout
Area variance: 0.0, Area: 896.5, Max Area: 896.5
Area variance: 0.0, Area: 827.0, Max Area: 827.0
Area variance: 0.0, Area: 653.5, Max Area: 653.5
Area variance: 0.0, Area: 477.5, Max Area: 477.5
Area variance: 1.0, Area: 0.0, Max Area: 477.5
Area variance: 0.0, Area: 446.0, Max Area: 446.0
Area variance: 0.0, Area: 482.0, Max Area: 482.0
Area variance: 0.0, Area: 494.0, Max Area: 494.0
Area variance: 0.0, Area: 814.5, Max Area: 814.5
Area variance: 0.0, Area: 499.5, Max Area: 499.5
Area variance: 0.0, Area: 384.0, Max Area: 384.0
Area variance: 0.0, Area: 710.5, Max Area: 710.5
Area variance: 0.0, Area: 363.5, Max Area: 363.5
Area variance: 0.0, Area: 621.0, Max Area: 621.0
Area variance: 0.0, Area: 627.5, Max Area: 627.5
Area variance: 0.0, Area: 676.0, Max Area: 676.0
Area variance: 0.0, Area: 580.5, Max Area: 580.5
Area variance: 0.0, Area: 520.0, Max Area: 520.0
Area variance: 0.0, Area: 533.0, Max Area: 533.0
Area variance: 0.0, Area: 335.0, Max Area: 335.0
Are